In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle


In [2]:
#a

In [3]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data2_smpl30_2/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data

def dtuji(data): 
    return uji(data) 


dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt


In [4]:
dtg =dtuji('gabung') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt

In [22]:
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2_smpl30_2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0.1'], axis=1)
        self.Y = df['0.1']
        self.dt=data

def dtlatih(data): 
    return latih(data) 
ddf9= dtlatih('latih_smoted30_1') #df1.X,df1.Y,df1.dt
ddu9= dtlatih('uji_smoted30_1') #df1.X,df1.Y,df1.dt

In [5]:

class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2_smpl30_2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0','1282','0.1'], axis=1)
        self.Y = df['0.1']
        self.dt=data

def dtlatih(data): 
    return latih(data) 
df1= dtlatih('latih_sampled30_1') #df1.X,df1.Y,df1.dt
df2= dtlatih('latih_sampled30_2') #df2.X,df2.Y,df2.dt
df3= dtlatih('latih_sampled30_3') #df3.X,df3.Y,df3.dt
df4= dtlatih('latih_sampled30_4') #df4.X,df4.Y,df4.dt
df5= dtlatih('latih_sampled30_5') #df5.X,df5.Y,df5.dt

du5=dtlatih('uji_sampled30_5') #df5.X,df5.Y,df5.dt
du4=dtlatih('uji_sampled30_4') #df5.X,df5.Y,df5.dt
du3=dtlatih('uji_sampled30_3') #df5.X,df5.Y,df5.dt
du2=dtlatih('uji_sampled30_2') #df5.X,df5.Y,df5.dt
du1=dtlatih('uji_sampled30_1') #df5.X,df5.Y,df5.dt

/home/biofarmaka/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
#df.X

In [7]:
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy','F-measure':'f1','Precision':'precision', 'Recall':'recall'}


In [8]:
from sklearn import svm

In [31]:
from sklearn import model_selection

In [9]:
def svm_(df):
    parameter_space0 = {
        'C': [0.03125, 0.5, 8, 128, 2048, 32768],
        'kernel': ['rbf'],
        'gamma' : [2, 0.125,  0.0078125,0.00048828125, 0.00003051757],
    }
    classifier0 = svm.SVC(class_weight='balanced',probability=True)
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=15, cv=5)
    classifier0.fit(df.X, df.Y)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data2_smpl30_2/model/svm_{0}'.format(df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [37]:
def svm_00(df):
    parameter_space0 = {
        'C': [0.03125, 0.5, 8, 128, 2048, 32768],
        'kernel': ['rbf'],
        'gamma' : [2, 0.125,  0.0078125,0.00048828125, 0.00003051757],
    }
    classifier0 = svm.SVC(class_weight='balanced',probability=True)
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=15, cv=5)
    
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(df.X, df.Y, test_size=0.9, random_state=0)
    
    classifier0.fit(X_train, Y_train)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data2_smpl30_2/model/svm_00_{0}'.format(df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [25]:
def report_rf(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_latih_sampled30_{1}'.format(model,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_{1}'.format(model,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def report_rf2(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_latih_smoted30_{1}'.format(model,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_{1}'.format(model,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model,p,data.dt,d))

def prediksi3(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_smoted30_{3}.xlsx'.format(model,p,data.dt,d))

def prediksi2(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.X))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(data.Y.value_counts())
    print(confusion_matrix(data.Y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    #pred[2] = data.com
    #pred[3] = data.prot
    print('Accuracy:',accuracy_score(data.Y, prob_x[1]))
    print('f1:',f1_score(data.Y, prob_x[1]))
    print('precision:',precision_score(data.Y, prob_x[1]))
    print('recall:',recall_score(data.Y, prob_x[1]))
    print('roc_auc:',roc_auc_score(data.Y, prob_x[1]))
    print('Confusion matrix:')
    print(confusion_matrix(data.Y, prob_x[1]))
    print('Results on the train set:')
    print(classification_report(data.Y, prob_x[1]))
    #print(pred.head())
    #print(prob_x)
    #pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    #pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [11]:
def save_svm(svm1,df):
    filename = '../data2_smpl30_2/model/svm_rbf_{0}'.format(df.dt)
    pickle.dump(svm1, open(filename, 'wb'))

In [33]:
def fitn3pred(model,p,datalatih,datauji1,datauji2,datauji3,d):
    svm_(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi2(md,model,p,datauji3,d)
def fitn3pred0(model,p,datalatih,datauji1,datauji2,datauji3,d):
    svm_00(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi2(md,model,p,datauji3,d)
def fitn2pred(model,p,datalatih,datauji1,datauji2,d):
    svm_(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
def fitnpred(model,p,datalatih,datauji1,d):
    svm_(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
def fitn3pred2(model,p,datalatih,datauji1,datauji2,datauji3,d):
    svm_(datalatih)
    md=report_rf2(model,datalatih,d)
    prediksi3(md,model,p,datauji1,d)
    prediksi3(md,model,p,datauji2,d)
    prediksi2(md,model,p,datauji3,d)

In [40]:
fitn3pred0('svm',0.5,df1,dtf,dtl,du1,1)

done_training
Best parameters found: {'C': 128, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
model file : svm_1
Best parameters found: {'C': 0.5, 'gamma': 2, 'kernel': 'rbf'}
CV RESULT :
AUC : 0.9998818639132765
Accuracy : 0.9844669407686372
F-measure : 0.9447299635369495
Precision : 0.9987730061349694
Recall : 0.9887381275440976
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[240]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[3039    9]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  uji_sampled30_1  : (0,1)
0    3537
1    3086
Name: 0.1, dtype: int64
[[3534    3]
 [   0 3086]]
Accuracy: 0.9995470330665861
f1: 0.9995141700404858
precision: 0.9990288119132406
recall: 1.0
roc_auc: 0.9995759117896523
Confusion matrix:
[[3534    3]
 [   0 3086]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3537
           1       1.00      1.00      1

In [ ]:
fitn3pred(svm,0.5,df1,dtf,dtl,du1,1)


In [14]:
md=report_rf('svm',df1,1)
prediksi(md,'svm',0.5,dtf,1)
prediksi(md,'svm',0.5,dtl,1)

model file : svm_1
Best parameters found: {'C': 0.5, 'gamma': 2, 'kernel': 'rbf'}
CV RESULT :
AUC : 0.9998818639132765
Accuracy : 0.9844669407686372
F-measure : 0.9447299635369495
Precision : 0.9987730061349694
Recall : 0.9887381275440976
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[240]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[3039    9]
 [   0    0]]


In [15]:
prediksi2(md,'svm',0.5,du1,1)

threshold 0.5 
model_hasilsmote_1 
hasil prediksi  uji_sampled30_1  : (0,1)
0    3537
1    3086
Name: 0.1, dtype: int64
[[3534    3]
 [   0 3086]]
Accuracy: 0.9995470330665861
f1: 0.9995141700404858
precision: 0.9990288119132406
recall: 1.0
roc_auc: 0.9995759117896523
Confusion matrix:
[[3534    3]
 [   0 3086]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3537
           1       1.00      1.00      1.00      3086

    accuracy                           1.00      6623
   macro avg       1.00      1.00      1.00      6623
weighted avg       1.00      1.00      1.00      6623



In [45]:
def svm_(df):
    parameter_space0 = {
        'C': [128], 'gamma': [0.0078125], 'kernel': ['rbf']
    }
    classifier0 = svm.SVC(class_weight='balanced',probability=True)
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=15, cv=5)
    classifier0.fit(df.X, df.Y)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data2_smpl30_2/model/svm_n_{0}'.format(df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [43]:
def report_rf(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_n_latih_sampled30_{1}'.format(model,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_n_{1}'.format(model,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def report_rf2(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_n_latih_smoted30_{1}'.format(model,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_n_{1}'.format(model,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_n_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model,p,data.dt,d))

def prediksi3(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_n_{1}p_{2}_latih_smoted30_{3}.xlsx'.format(model,p,data.dt,d))

def prediksi2(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.X))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(data.Y.value_counts())
    print(confusion_matrix(data.Y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    #pred[2] = data.com
    #pred[3] = data.prot
    print('Accuracy:',accuracy_score(data.Y, prob_x[1]))
    print('f1:',f1_score(data.Y, prob_x[1]))
    print('precision:',precision_score(data.Y, prob_x[1]))
    print('recall:',recall_score(data.Y, prob_x[1]))
    print('roc_auc:',roc_auc_score(data.Y, prob_x[1]))
    print('Confusion matrix:')
    print(confusion_matrix(data.Y, prob_x[1]))
    print('Results on the train set:')
    print(classification_report(data.Y, prob_x[1]))
    #print(pred.head())
    #print(prob_x)
    #pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    #pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [ ]:
fitn3pred('svm',0.5,df1,dtf,dtl,du1,1)
fitn3pred('svm',0.5,df2,dtf,dtl,du2,2)
fitn3pred('svm',0.5,df3,dtf,dtl,du3,3)
fitn3pred('svm',0.5,df4,dtf,dtl,du4,4)
fitn3pred('svm',0.5,df5,dtf,dtl,du5,5)

In [29]:
md=report_rf2('svm',ddf9,1)
prediksi(md,'svm',0.5,dtf,1)
prediksi(md,'svm',0.5,dtl,1)


model file : svm_1
Best parameters found: {'C': 0.5, 'gamma': 2, 'kernel': 'rbf'}
CV RESULT :
AUC : 0.9981154492654151
Accuracy : 0.8587792725304386
F-measure : 0.8265322440049723
Precision : 0.997494711475194
Recall : 0.705698778833107
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[240]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[3039    9]
 [   0    0]]


NameError: name 'ddu' is not defined

In [30]:
prediksi2(md,'svm',0.5,ddu9,1)

threshold 0.5 
model_hasilsmote_1 
hasil prediksi  uji_smoted30_1  : (0,1)
0    3537
1    3086
Name: 0.1, dtype: int64
[[3534    3]
 [ 197 2889]]
Accuracy: 0.9698022044390759
f1: 0.9665439946470392
precision: 0.9989626556016598
recall: 0.9361633182112767
roc_auc: 0.9676575708952907
Confusion matrix:
[[3534    3]
 [ 197 2889]]
Results on the train set:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3537
           1       1.00      0.94      0.97      3086

    accuracy                           0.97      6623
   macro avg       0.97      0.97      0.97      6623
weighted avg       0.97      0.97      0.97      6623



In [20]:
fitn3pred('svm',0.5,df2,dtf,dtl,du2,2)
fitn3pred('svm',0.5,df3,dtf,dtl,du3,3)
fitn3pred('svm',0.5,df4,dtf,dtl,du4,4)
fitn3pred('svm',0.5,df5,dtf,dtl,du5,5)

done_training
Best parameters found: {'C': 0.5, 'gamma': 2, 'kernel': 'rbf'}
done_savemodel
model file : svm_2
Best parameters found: {'C': 0.5, 'gamma': 2, 'kernel': 'rbf'}
CV RESULT :
AUC : 0.9999251036383562
Accuracy : 0.9989644222342975
F-measure : 0.9989007673244013
Precision : 0.9990383644049103
Recall : 0.9987640190968949
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbafika  : (0,1)
[[240]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbalinda  : (0,1)
[[3039    9]
 [   0    0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  uji_sampled30_2  : (0,1)
0    3450
1    3173
Name: 0.1, dtype: int64
[[3445    5]
 [   4 3169]]
Accuracy: 0.9986410991997584
f1: 0.9985820072475184
precision: 0.9984247006931317
recall: 0.9987393633785061
roc_auc: 0.9986450440080935
Confusion matrix:
[[3445    5]
 [   4 3169]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3450
           1       1.00      1.00

In [ ]:
svma=svm_cg(df1,'rbf',0.5, 0.0078125)
svmb=svm_cg(df2,'rbf',8, 0.0078125)
svmc=svm_cg(df3,'rbf', 0.5,  0.0078125)
svmd=svm_cg(df4,'rbf',8, 0.0078125)
svme=svm_cg(df5,'rbf',0.5, 0.0078125)

In [13]:
svme=svm_cg(df5,'rbf',0.5, 0.0078125)

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [57]:
save_svm(svm1,df1)
save_svm(svm2,df2)
save_svm(svm3,df3)
save_svm(svm4,df4)
save_svm(svm5,df5)

In [ ]:
svm1=svm_(df1,'rbf')
svm2=svm_(df2,'rbf')
svm3=svm_(df3,'rbf')
svm4=svm_(df4,'rbf')
svm5=svm_(df5,'rbf')

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [12]:
svm5=svm_(df5,'rbf')

NameError: name 'svm_' is not defined

In [90]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results_svm(o):
    rf = pickle.load(open('../data2_smpl30_2/model/svm_rbf_{0}'.format(o.dt), 'rb'))
    print('svm_rbf_{0}'.format(o.dt))
    print('Best parameters found:', rf.best_params_)
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])

In [91]:
print_cv_results_svm(df1)
print_cv_results_svm(df2)
print_cv_results_svm(df3)
print_cv_results_svm(df4)
print_cv_results_svm(df5)

svm_rbf_hasilsmote_1
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9746857896464474
Accuracy : 0.918904958677686
F-measure : 0.9078592977812351
Precision : 1.0
Recall : 0.9090909090909091
svm_rbf_hasilsmote_2
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9748604308446257
Accuracy : 0.9231663223140496
F-measure : 0.9114358089683635
Precision : 1.0
Recall : 0.9083161157024794
svm_rbf_hasilsmote_3
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9699539313202955
Accuracy : 0.9258780991735537
F-measure : 0.9145832117684626
Precision : 1.0
Recall : 0.9108987603305785
svm_rbf_hasilsmote_4
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9683368448861527
Accuracy : 0.921745867768595
F-measure : 0.911005668876379
Precision : 1.0
Recall : 0.9096074380165289
svm_rbf_hasilsmote_5
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9766685774405033
Ac

In [19]:

def prediksi(md,model,n,m,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_modelhasilsmote{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [23]:
def fitn2pred(model,n,m,p,datalatih,datauji1,datauji2,datauji3,d):
    md=print_cv_results(model,n,m,datalatih)
    prediksi(md,model,n,m,p,datauji1,d)
    prediksi(md,model,n,m,p,datauji2,d)
    prediksi(md,model,n,m,p,datauji3,d)

In [25]:
#fitn2pred(mlp,100,1000,0.5,df1,dtf,dtl,dtg,1)